<a href="https://colab.research.google.com/github/ananyaarya02/TicketMind/blob/main/Multiclassifier_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
import os
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import re
import joblib
import string
string.punctuation
import re
import nltk
import pandas as pd
import html
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.tokenize import word_tokenize



In [ ]:
path = kagglehub.dataset_download("tobiasbueck/multilingual-customer-support-tickets")
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14


In [ ]:
subfolders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

print("Subfolders:", subfolders)
print("Files:", files)


Subfolders: []
Files: ['dataset-tickets-multi-lang3-4k.csv', 'dataset-tickets-german_normalized_50_5_2.csv', 'dataset-tickets-german_normalized.csv', 'dataset-tickets-multi-lang-4-20k.csv', 'aa_dataset-tickets-multi-lang-5-2-50-version.csv']


In [ ]:
data="/root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14/dataset-tickets-multi-lang-4-20k.csv"
df = pd.read_csv(data)

In [ ]:
df.head(5)

,subject,body,answer,type,queue,priority,language,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Unvorhergesehener Absturz der Datenanalyse-Pla...,Die Datenanalyse-Plattform brach unerwartet ab...,Ich werde Ihnen bei der Lösung des Problems he...,Incident,General Inquiry,low,de,Crash,Technical,Bug,Hardware,Resolution,Outage,Documentation,NaN
1,Customer Support Inquiry,Seeking information on digital strategies that...,We offer a variety of digital strategies and s...,Request,Customer Service,medium,en,Feedback,Sales,IT,Tech Support,NaN,NaN,NaN,NaN
2,Data Analytics for Investment,I am contacting you to request information on ...,I am here to assist you with data analytics to...,Request,Customer Service,medium,en,Technical,Product,Guidance,Documentation,Performance,Feature,NaN,NaN
3,Krankenhaus-Dienstleistung-Problem,Ein Medien-Daten-Sperrverhalten trat aufgrund ...,Zurück zur E-Mail-Beschwerde über den Sperrver...,Incident,Customer Service,high,de,Security,Breach,Login,Maintenance,Incident,Resolution,Feedback,NaN
4,Security,"Dear Customer Support, I am reaching out to in...","Dear [name], we take the security of medical d...",Request,Customer Service,medium,en,Security,Customer,Compliance,Breach,Documentation,Guidance,NaN,NaN


In [ ]:
df=df.drop("tag_2",axis=1)
df=df.drop("tag_3",axis=1)
df=df.drop("tag_4",axis=1)
df=df.drop("tag_5",axis=1)
df=df.drop("tag_6",axis=1)
df=df.drop("tag_7",axis=1)
df=df.drop("tag_8",axis=1)
df=df.drop("answer",axis=1)


In [ ]:
df['subject'] = df['subject'].fillna('')
df['body'] = df['body'].fillna('')

In [ ]:
df.drop(df[df['language'] == 'de'].index, inplace=True)

In [ ]:
df=df.drop("language",axis=1)

In [ ]:
#remove html tags
def clean_html(text):
    return BeautifulSoup(text, "html.parser").get_text()
df['subject'] = df['subject'].fillna('').apply(clean_html)
df['body'] = df['body'].fillna('').apply(clean_html)

In [ ]:


def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

df['subject']= df['subject'].apply(lambda x:remove_punctuation(x))
df['body']= df['body'].apply(lambda x:remove_punctuation(x))
df.head()

,subject,body,type,queue,priority,tag_1
1,Customer Support Inquiry,Seeking information on digital strategies that...,Request,Customer Service,medium,Feedback
2,Data Analytics for Investment,I am contacting you to request information on ...,Request,Customer Service,medium,Technical
4,Security,Dear Customer Support I am reaching out to inq...,Request,Customer Service,medium,Security
5,Concerns About Securing Medical Data on 2in1 C...,Inquiring about best practices for securing me...,Request,Technical Support,medium,Security
7,Problem with Integration,The integration stopped working unexpectedly c...,Problem,IT Support,high,Technical


In [ ]:
df[['subject', 'body']] = df[['subject', 'body']].apply(lambda x: x.str.lower())

In [ ]:

def tokenization(text):
    if isinstance(text, str):
        tokens = re.split('\W+', text)
        return tokens
    return []

df[['subject', 'body']] = df[['subject', 'body']].applymap(tokenization)
df.head()

<>:5: SyntaxWarning: invalid escape sequence '\W'
<>:5: SyntaxWarning: invalid escape sequence '\W'
/tmp/ipython-input-1900022494.py:5: SyntaxWarning: invalid escape sequence '\W'
  tokens = re.split('\W+', text)
/tmp/ipython-input-1900022494.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['subject', 'body']] = df[['subject', 'body']].applymap(tokenization)


,subject,body,type,queue,priority,tag_1
1,"[customer, support, inquiry]","[seeking, information, on, digital, strategies...",Request,Customer Service,medium,Feedback
2,"[data, analytics, for, investment]","[i, am, contacting, you, to, request, informat...",Request,Customer Service,medium,Technical
4,[security],"[dear, customer, support, i, am, reaching, out...",Request,Customer Service,medium,Security
5,"[concerns, about, securing, medical, data, on,...","[inquiring, about, best, practices, for, secur...",Request,Technical Support,medium,Security
7,"[problem, with, integration]","[the, integration, stopped, working, unexpecte...",Problem,IT Support,high,Technical


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import nltk
import pandas as pd
import html
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.tokenize import word_tokenize

In [ ]:

stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output
df[['subject', 'body']]= df[['subject', 'body']].apply(lambda x:remove_stopwords(x))


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()

# POS tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return 'a'  # Adjective
    elif tag.startswith('V'):
        return 'v'  # Verb
    elif tag.startswith('N'):
        return 'n'  # Noun
    elif tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'  # Default to noun if not recognized

# lemmatize
def lemmatize_with_estimated_pos(tokens):
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatized_tokens = []
    for token, tag in tagged_tokens:
        pos = get_wordnet_pos(tag)
        lemma = lemmatizer.lemmatize(token, pos=pos)
        lemmatized_tokens.append(lemma)
    return lemmatized_tokens

for i, row in df.iterrows():
    df.at[i, 'subject'] = lemmatize_with_estimated_pos(row['subject'])
    df.at[i, 'body'] = lemmatize_with_estimated_pos(row['body'])


In [ ]:
subject_vectorizer = TfidfVectorizer(
    max_features=5000,
)

body_vectorizer = TfidfVectorizer(
    max_features=20000,
)

In [ ]:
x_sub_train, x_sub_test, x_body_train, x_body_test = train_test_split(
    df['subject'], df['body'], test_size=0.2, random_state=42)

In [ ]:
x_sub_train = [" ".join(tokens) for tokens in x_sub_train]
x_sub_test  = [" ".join(tokens) for tokens in x_sub_test]

x_body_train = [" ".join(tokens) for tokens in x_body_train]
x_body_test  = [" ".join(tokens) for tokens in x_body_test]


In [ ]:
x_sub_train_v = subject_vectorizer.fit_transform(x_sub_train)
x_sub_test_v  = subject_vectorizer.transform(x_sub_test)

x_body_train_v = body_vectorizer.fit_transform(x_body_train)
x_body_test_v = body_vectorizer.transform(x_body_test)

In [ ]:
x_train = hstack([x_sub_train_v, x_body_train_v])
x_test  = hstack([x_sub_test_v, x_body_test_v])

In [ ]:
l_type = LabelEncoder()
l_queue = LabelEncoder()
l_priority = LabelEncoder()

Y_type = l_type.fit_transform(df['type'])
Y_queue = l_queue.fit_transform(df['queue'])
Y_priority = l_priority.fit_transform(df['priority'])

In [ ]:
Y = np.vstack([Y_type, Y_queue, Y_priority]).T

In [ ]:
Y_train, Y_test = train_test_split(
    Y,
    test_size=0.2,
    random_state=42
)

In [ ]:
xgb = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

multi_models = MultiOutputClassifier(xgb, n_jobs=-1)
multi_models.fit(x_train, Y_train)
preds = multi_models.predict(x_test)
print("Type accuracy:", accuracy_score(Y_test[:,0], preds[:,0]))
print("Queue accuracy:", accuracy_score(Y_test[:,1], preds[:,1]))
print("Priority accuracy:", accuracy_score(Y_test[:,2], preds[:,2]))

Type accuracy: 0.8150943396226416
Queue accuracy: 0.4830188679245283
Priority accuracy: 0.5622641509433962


In [ ]:
log_reg = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    class_weight='balanced'
)

model = MultiOutputClassifier(log_reg)
model.fit(x_train, Y_train)
ypred = model.predict(x_test)

In [ ]:
print("Type accuracy:", accuracy_score(Y_test[:,0], ypred[:,0]))
print("Queue accuracy:", accuracy_score(Y_test[:,1], ypred[:,1]))
print("Priority accuracy:", accuracy_score(Y_test[:,2], ypred[:,2]))

Type accuracy: 0.810482180293501
Queue accuracy: 0.3631027253668763
Priority accuracy: 0.5039832285115304


In [ ]:
joblib.dump(multi_models, "multiclassifier_model1.pkl")
joblib.dump(subject_vectorizer, "tfidf_subject1.pkl")
joblib.dump(body_vectorizer, "tfidf_body1.pkl")

['tfidf_body1.pkl']